# 1. Introducción a DataFrames:
## Conceptos básicos:
Los DataFrames son una abstracción de datos estructurados, organizados en filas y columnas, con un esquema definido. Esta estructura facilita la manipulación y el análisis de datos utilizando las APIs de Spark SQL.

## Creación de DataFrames:
- **Desde RDDs**: Los DataFrames pueden crearse a partir de RDDs (colecciones distribuidas de datos). La creación de un DataFrame desde un RDD permite trabajar con datos no estructurados transformándolos en un formato tabular.
- **Desde archivos**: Spark SQL permite la creación de DataFrames desde varios formatos de archivos, como CSV, JSON y Parquet. Puedes cargar estos archivos directamente en un DataFrame utilizando la API de Spark SQL. También se pueden usar otros formatos de archivo.
- **Desde tablas Hive**: Puedes crear DataFrames a partir de tablas existentes en Hive, aprovechando el metastore de Hive.
- **Otras fuentes**: Spark puede leer datos de diversas fuentes incluyendo bases de datos relacionales mediante JDBC, NoSQL, ORC, y otros sistemas de almacenamiento.


### 1. Cargar datos desde un RDD:
Para convertir un RDD en un DataFrame, se utiliza la función `toDF()` o `createDataFrame()`.

- **toDF()**: Infiere el esquema del DataFrame a partir del RDD, normalmente usado con una tupla o lista en Python.  https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.toDF.html (similar, la original no está documentada en https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)
- **createDataFrame()**: Permite especificar explícitamente el esquema (`StructType`) del DataFrame.  https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.createDataFrame.html

In [0]:
# Inicializamos sesión

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RDDtoDF").getOrCreate()


In [0]:
import sys
print("Python version: ", sys.version)

Python version:  3.9.21 (main, Dec  4 2024, 08:53:34) 
[GCC 9.4.0]


In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print("Spark version: ", sc.version)


Spark version:  3.3.2


In [0]:
## Ejemplo con toDF():
rdd = spark.sparkContext.parallelize([("Alice", 34), ("Bob", 23)])
df = rdd.toDF(["name", "age"])
df.show()

## Ejemplo sobre rdd cargado de un fichero (error porque no consigue inferir el esquema)
# lines = sc.textFile("dbfs:/FileStore/u.data")
df_peliculas = spark.read.option("header", "false").csv("dbfs:/FileStore/u.data")
# df_peliculas = lines.toDF()
df_peliculas.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 34|
|  Bob| 23|
+-----+---+

+--------------------+
|                 _c0|
+--------------------+
|196\t242\t3\t8812...|
|186\t302\t3\t8917...|
|22\t377\t1\t87888...|
|244\t51\t2\t88060...|
|166\t346\t1\t8863...|
|298\t474\t4\t8841...|
|115\t265\t2\t8811...|
|253\t465\t5\t8916...|
|305\t451\t3\t8863...|
| 6\t86\t3\t883603013|
|62\t257\t2\t87937...|
|286\t1014\t5\t879...|
|200\t222\t5\t8760...|
|210\t40\t3\t89103...|
|224\t29\t3\t88810...|
|303\t785\t3\t8794...|
|122\t387\t5\t8792...|
|194\t274\t2\t8795...|
|291\t1042\t4\t874...|
|234\t1184\t2\t892...|
+--------------------+
only showing top 20 rows



In [0]:
## Ejemplo con createDataFrame() especificando el esquema:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

esquema = StructType([
            StructField("usuario",IntegerType(),False),
            StructField("pelicula",IntegerType(),False),
            StructField("rating", IntegerType(),False),
            StructField("timestamp",IntegerType(),False)
])

lineas_rdd = sc.textFile("dbfs:/FileStore/u.data")

# Convertimos un rdd de filas en un rdd de listas de 4 strings
lineas_rdd_cadenas = lineas_rdd.map(lambda x: x.split())

# Como el esquema espera enteros, convertimos el rdd a listas de 4 enteros
lineas_rdd_enteros = lineas_rdd_cadenas.map(lambda x: [int(x[0]), int(x[1]), int(x[2]), int(x[3])])

df_peliculas = spark.createDataFrame(lineas_rdd_enteros,esquema)
df_peliculas.show()

+-------+--------+------+---------+
|usuario|pelicula|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



In [0]:
# Mismo caso pero definiendo la función para el map
def leerFila(fila):
    lista = fila.split()
    return [int(lista[0]),int(lista[1]),int(lista[2]),int(lista[3])]

lineas_rdd_filas = sc.textFile("dbfs:/FileStore/u.data")
lineas_rdd_enteros2 = lineas_rdd_filas.map(leerFila)

df_peliculas2 = spark.createDataFrame(lineas_rdd_enteros2,esquema)
df_peliculas2.show()

+-------+--------+------+---------+
|usuario|pelicula|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



### 2. Cargar datos desde ficheros CSV:

#### Sintaxis:
Se utiliza `spark.read.csv()`. Se pueden especificar opciones como `header` para indicar si el archivo tiene encabezado e `inferSchema` para que Spark infiera los tipos de datos.

- `header` indica si la primera línea del archivo CSV contiene los nombres de las columnas.
- `inferSchema` permite a Spark determinar automáticamente los tipos de datos de cada columna.

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.csv.html

#### Consideraciones
- **Rutas de archivos**: Asegúrate de proporcionar las rutas correctas a tus archivos CSV y Parquet.
- **Esquema**: Si no se utiliza `inferSchema` al leer archivos CSV, el esquema del DataFrame debe especificarse explícitamente.
- **DataFrames**: Los DataFrames proporcionan una forma de procesar y analizar datos estructurados. A diferencia de los RDDs, los DataFrames están basados en un esquema, es decir, conocen los nombres y tipos de las columnas de un conjunto de datos.

In [0]:
# Desde un fichero CSV
df=spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/olive.csv")
df.show()


+-------+----+----------------+--------------------+-------------+-------+---------------------+-------------------+-------+---------------------+----------+------------------+------------+
|Country|Year|Beginning Stocks|Domestic Consumption|Ending Stocks|Exports|Feed Waste Dom. Cons.|Food Use Dom. Cons.|Imports|Industrial Dom. Cons.|Production|Total Distribution|Total Supply|
+-------+----+----------------+--------------------+-------------+-------+---------------------+-------------------+-------+---------------------+----------+------------------+------------+
|Algeria|1964|               0|                  15|            0|      3|                    0|                 15|      0|                    0|        18|                18|          18|
|Algeria|1965|               0|                  12|            0|      5|                    0|                 12|      0|                    0|        17|                17|          17|
|Algeria|1966|               0|                  1

### 3. Cargar datos desde ficheros Parquet:
Se utiliza `spark.read.parquet()`.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.parquet.html  

#### Consideraciones
- **Esquema**: El esquema se almacena en el mismo archivo.

In [0]:
# Desde un único fichero parquet
df=spark.read.parquet("dbfs:/FileStore/palm.parquet")
df.show(1000)


+------------------+----+--------------+----------------+--------------------+-------------+-------+---------------------+-------------------+-------+---------------------+----------+------------------+------------+-----+
|           Country|Year|Area Harvested|Beginning Stocks|Domestic Consumption|Ending Stocks|Exports|Feed Waste Dom. Cons.|Food Use Dom. Cons.|Imports|Industrial Dom. Cons.|Production|Total Distribution|Total Supply|Yield|
+------------------+----+--------------+----------------+--------------------+-------------+-------+---------------------+-------------------+-------+---------------------+----------+------------------+------------+-----+
|       Afghanistan|1964|           0.0|             0.0|                 0.0|          0.0|    0.0|                  0.0|                0.0|    0.0|                  0.0|       0.0|               0.0|         0.0|  0.0|
|       Afghanistan|1965|           0.0|             0.0|                 0.0|          0.0|    0.0|            

# 2. API de DataFrames:
Para realizar transformaciones en un DataFrame en Spark con Python, se utilizan diversas funciones que permiten modificar, seleccionar, o agregar datos. Esta es la sintaxis y ejemplos de uso de algunas de las transformaciones más comunes:


## Transformaciones:
- **select**: Permite seleccionar columnas específicas de un DataFrame.
- **filter**: Permite filtrar filas basadas en una condición.
- **withColumn**: Permite añadir nuevas columnas o modificar las existentes.
- **Otras transformaciones**: La API incluye otras transformaciones para manipular los datos como groupBy, sort y join. También permite crear funciones definidas por el usuario para manipulación personalizada de datos.

## Acciones:
- **show**: Muestra las primeras filas de un DataFrame.
- **count**: Cuenta el número de filas en un DataFrame.
- **collect**: Retorna todos los elementos de un DataFrame al driver (cuidado con el uso en grandes datasets).
- **Otras acciones**: Incluyen take, takeSample y describe para obtener información y estadísticas sobre los DataFrames.

## Consideraciones:

- **Inmutabilidad**: Los DataFrames son inmutables; cada transformación crea un nuevo DataFrame.
- **show()**: La función `show()` se utiliza para mostrar una muestra de los datos resultantes tras una transformación.
- **Importaciones**: Algunas funciones requieren importaciones adicionales desde `pyspark.sql.functions`, como `col`, `lit`, `expr`, `avg`, `count`, etc.
- **Expresiones SQL**: Puedes usar expresiones SQL con `expr()` y `selectExpr()` para transformaciones más complejas.
- **Columnas**: Las columnas se pueden referenciar usando su nombre como string, usando la notación de corchetes sobre el DataFrame o con la función `col()`.

### 1. select():
Se utiliza para seleccionar un subconjunto de columnas de un DataFrame. También se puede usar `selectExpr()` para seleccionar columnas con expresiones SQL.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.select.html

In [0]:
df=spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/olive.csv")

df.select("Country", "Year", "Production").show() 
df.select('*').show()
df.select(df.Country, (df.Production * 1000).alias('Production (Tm)')).show()


+-------+----+----------+
|Country|Year|Production|
+-------+----+----------+
|Algeria|1964|        18|
|Algeria|1965|        17|
|Algeria|1966|        16|
|Algeria|1967|        22|
|Algeria|1968|        18|
|Algeria|1969|        22|
|Algeria|1970|        13|
|Algeria|1971|        23|
|Algeria|1972|        15|
|Algeria|1973|        16|
|Algeria|1974|         8|
|Algeria|1975|        18|
|Algeria|1976|        15|
|Algeria|1977|         5|
|Algeria|1978|        14|
|Algeria|1979|        10|
|Algeria|1980|        18|
|Algeria|1981|        15|
|Algeria|1982|        16|
|Algeria|1983|        13|
+-------+----+----------+
only showing top 20 rows

+-------+----+----------------+--------------------+-------------+-------+---------------------+-------------------+-------+---------------------+----------+------------------+------------+
|Country|Year|Beginning Stocks|Domestic Consumption|Ending Stocks|Exports|Feed Waste Dom. Cons.|Food Use Dom. Cons.|Imports|Industrial Dom. Cons.|Production|Tot

### 2. filter() o where():
Se utiliza para filtrar filas basadas en una condición. `filter()` y `where()` son sinónimos.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html

In [0]:
df.filter((df["Country"] == "Spain") & (df["Year"] < 1984)).select("Country", "Year", "Production").show()


+-------+----+----------+
|Country|Year|Production|
+-------+----+----------+
|  Spain|1964|       110|
|  Spain|1965|       324|
|  Spain|1966|       437|
|  Spain|1967|       259|
|  Spain|1968|       480|
|  Spain|1969|       358|
|  Spain|1970|       475|
|  Spain|1971|       341|
|  Spain|1972|       440|
|  Spain|1973|       447|
|  Spain|1974|       333|
|  Spain|1975|       455|
|  Spain|1976|       390|
|  Spain|1977|       361|
|  Spain|1978|       500|
|  Spain|1979|       433|
|  Spain|1980|       479|
|  Spain|1981|       300|
|  Spain|1982|       666|
|  Spain|1983|       258|
+-------+----+----------+



### 3. withColumn():
Se utiliza para añadir una nueva columna o reemplazar una existente. La función `lit()` crea una columna con un valor literal y `expr()` permite usar expresiones SQL.
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html

In [0]:
from pyspark.sql.functions import lit, expr
from datetime import datetime

df=spark.read.option("header", "true").option("inferSchema", "true").csv("dbfs:/FileStore/olive.csv")
df_Spain = df.filter((df["Country"] == "Spain") | (df["Country"] == "European Union")).select("Country", "Year", "Production")

df_Spain = df_Spain.withColumn("Production (Tm)", df_Spain["Production"]*1000)
df_Spain = df_Spain.withColumn("Region", expr("CASE WHEN Year <= 1990 THEN 'Spain' ELSE 'EU' END"))

current_year = datetime.now().year
df_Spain = df_Spain.withColumn("Diff_Years", lit(current_year) - df_Spain["Year"])

df_Spain.show(df_Spain.count())

+--------------+----+----------+---------------+------+----------+
|       Country|Year|Production|Production (Tm)|Region|Diff_Years|
+--------------+----+----------+---------------+------+----------+
|European Union|1964|         0|              0| Spain|        61|
|European Union|1965|         0|              0| Spain|        60|
|European Union|1966|         0|              0| Spain|        59|
|European Union|1967|         0|              0| Spain|        58|
|European Union|1968|         0|              0| Spain|        57|
|European Union|1969|         0|              0| Spain|        56|
|European Union|1970|         0|              0| Spain|        55|
|European Union|1971|         0|              0| Spain|        54|
|European Union|1972|         0|              0| Spain|        53|
|European Union|1973|         0|              0| Spain|        52|
|European Union|1974|         0|              0| Spain|        51|
|European Union|1975|         0|              0| Spain|       

### 4. withColumnRenamed():
Se utiliza para renombrar una columna existente.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html

In [0]:
df_Spain.withColumnRenamed("Country", "País").withColumnRenamed("Year", "Año").show()

+--------------+----+----------+---------------+------+----------+
|          País| Año|Production|Production (Tm)|Region|Diff_Years|
+--------------+----+----------+---------------+------+----------+
|European Union|1964|         0|              0| Spain|        61|
|European Union|1965|         0|              0| Spain|        60|
|European Union|1966|         0|              0| Spain|        59|
|European Union|1967|         0|              0| Spain|        58|
|European Union|1968|         0|              0| Spain|        57|
|European Union|1969|         0|              0| Spain|        56|
|European Union|1970|         0|              0| Spain|        55|
|European Union|1971|         0|              0| Spain|        54|
|European Union|1972|         0|              0| Spain|        53|
|European Union|1973|         0|              0| Spain|        52|
|European Union|1974|         0|              0| Spain|        51|
|European Union|1975|         0|              0| Spain|       

### 5. groupBy():
Se utiliza para agrupar filas con valores iguales en una columna y realizar operaciones de agregación. Se combina con funciones de agregación como `count()`, `sum()`, `avg()`, `min()`, `max()`.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.groupBy.html

In [0]:
# Podemos realizarlo de dos formas: utilizando funciones de F o con diccionarios. 
# La primera es más clara y permite realizar varias agregaciones sobre la misma columna.

df=spark.read.parquet("dbfs:/FileStore/palm.parquet")

from pyspark.sql import functions as F
df.select(df.Country, df.Year, df.Production) \
    .groupBy("Country") \
    .agg(
        F.sum("Production").alias("TotalProd"),
        F.max("Production").alias("MaxProd")
    ) \
    .orderBy("TotalProd",ascending=False) \
.show(4)

df.select(df.Country, df.Year, df.Production) \
    .groupBy("Country") \
    .agg({"Production": "sum"}) \
    .withColumnRenamed("sum(Production)", "TotalProd") \
    .orderBy("TotalProd", ascending=False) \
.show(4)

+---------+---------+-------+
|  Country|TotalProd|MaxProd|
+---------+---------+-------+
|Indonesia| 721653.0|47000.0|
| Malaysia| 552873.0|20800.0|
| Thailand|  49854.0| 3450.0|
|  Nigeria|  43015.0| 1500.0|
+---------+---------+-------+
only showing top 4 rows

+---------+---------+
|  Country|TotalProd|
+---------+---------+
|Indonesia| 721653.0|
| Malaysia| 552873.0|
| Thailand|  49854.0|
|  Nigeria|  43015.0|
+---------+---------+
only showing top 4 rows



### 6. sort() o orderBy():
Se utiliza para ordenar las filas del DataFrame. `sort()` y `orderBy()` son equivalentes y pueden usar el orden ascendente (`asc`) o descendente (`desc`).  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.sort.html  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.orderBy.html

In [0]:
from pyspark.sql import functions as F
df=spark.read.parquet("dbfs:/FileStore/palm.parquet")
df = df.select(df.Country, df.Year, df.Production).groupBy("Country","Year").agg(F.sum("Production").alias("TotalProd"))
df.sort("Country").show(5)
df.sort(df["Country"].desc()).show(5)
df.orderBy("TotalProd").show(5)
df.orderBy(df["TotalProd"].desc()).show(5)

+-----------+----+---------+
|    Country|Year|TotalProd|
+-----------+----+---------+
|Afghanistan|2022|      0.0|
|Afghanistan|2021|      0.0|
|Afghanistan|1999|      0.0|
|Afghanistan|2009|      0.0|
|Afghanistan|2012|      0.0|
+-----------+----+---------+
only showing top 5 rows

+--------+----+---------+
| Country|Year|TotalProd|
+--------+----+---------+
|Zimbabwe|1965|      0.0|
|Zimbabwe|1964|      0.0|
|Zimbabwe|1991|      0.0|
|Zimbabwe|1978|      0.0|
|Zimbabwe|2013|      0.0|
+--------+----+---------+
only showing top 5 rows

+-----------------+----+---------+
|          Country|Year|TotalProd|
+-----------------+----+---------+
|      El Salvador|1975|      0.0|
|            Italy|2023|      0.0|
|Former Yugoslavia|1971|      0.0|
|            Haiti|2012|      0.0|
|          Ireland|1976|      0.0|
+-----------------+----+---------+
only showing top 5 rows

+---------+----+---------+
|  Country|Year|TotalProd|
+---------+----+---------+
|Indonesia|2023|  47000.0|
|Indone

### 7. drop():
Se utiliza para eliminar una o varias columnas del DataFrame.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.drop.html

In [0]:
df_Spain.show(5)
df_Spain_reducido = df_Spain.drop("Year", "Diff_Years")
df_Spain_reducido.show(5)

+--------------+----+----------+---------------+------+----------+
|       Country|Year|Production|Production (Tm)|Region|Diff_Years|
+--------------+----+----------+---------------+------+----------+
|European Union|1964|         0|              0| Spain|        61|
|European Union|1965|         0|              0| Spain|        60|
|European Union|1966|         0|              0| Spain|        59|
|European Union|1967|         0|              0| Spain|        58|
|European Union|1968|         0|              0| Spain|        57|
+--------------+----+----------+---------------+------+----------+
only showing top 5 rows

+--------------+----------+---------------+------+
|       Country|Production|Production (Tm)|Region|
+--------------+----------+---------------+------+
|European Union|         0|              0| Spain|
|European Union|         0|              0| Spain|
|European Union|         0|              0| Spain|
|European Union|         0|              0| Spain|
|European Union

### 8. distinct():
Se utiliza para eliminar las filas duplicadas del DataFrame.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.distinct.html

In [0]:
df_Spain_reducido.distinct().show()


+--------------+----------+---------------+------+
|       Country|Production|Production (Tm)|Region|
+--------------+----------+---------------+------+
|European Union|         0|              0|    EU|
|European Union|      2188|        2188000|    EU|
|European Union|      2500|        2500000|    EU|
|European Union|      2110|        2110000|    EU|
|European Union|      2324|        2324000|    EU|
|European Union|      1944|        1944000|    EU|
|European Union|      2025|        2025000|    EU|
|European Union|      2350|        2350000|    EU|
|European Union|      2483|        2483000|    EU|
|European Union|         0|              0| Spain|
|European Union|      1435|        1435000|    EU|
|European Union|      2132|        2132000|    EU|
|European Union|      2415|        2415000|    EU|
|European Union|      2700|        2700000|    EU|
|European Union|      2235|        2235000|    EU|
|European Union|      2402|        2402000|    EU|
|European Union|      1752|    

### 9. join():
Se utiliza para combinar dos DataFrames basados en una o varias columnas en común. Se puede especificar el tipo de join: 'inner', 'outer', 'left_outer', 'right_outer', o 'leftsemi'.  
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.join.html

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import functions as F

# Definir el esquema del DataFrame de ratings
esquemaRatings = StructType([
    StructField("UserID", IntegerType(), True),
    StructField("MovieID", IntegerType(), True),
    StructField("Rating", IntegerType(), True),
    StructField("Timestamp", IntegerType(), True)
])

# Cargar el archivo de ratings u-data en un DataFrame, con separador el tabulador \t
dfRatings = spark.read.csv("dbfs:/FileStore/u.data", sep="\t", schema=esquemaRatings, header=False)

# Definir esquema para el DataFrame de películas
esquemaPeliculas = StructType([
    StructField("MovieID", IntegerType(), True),
    StructField("Title", StringType(), True),
    StructField("ReleaseDate", StringType(), True),
    StructField("EmptyColumn", StringType(), True),
    StructField("IMDB_URL", StringType(), True),
    StructField("Unknown", IntegerType(), True),
    StructField("Action", IntegerType(), True),
    StructField("Adventure", IntegerType(), True),
    StructField("Animation", IntegerType(), True),
    StructField("Children", IntegerType(), True),
    StructField("Comedy", IntegerType(), True),
    StructField("Crime", IntegerType(), True),
    StructField("Documentary", IntegerType(), True),
    StructField("Drama", IntegerType(), True),
    StructField("Fantasy", IntegerType(), True),
    StructField("FilmNoir", IntegerType(), True),
    StructField("Horror", IntegerType(), True),
    StructField("Musical", IntegerType(), True),
    StructField("Mystery", IntegerType(), True),
    StructField("Romance", IntegerType(), True),
    StructField("SciFi", IntegerType(), True),
    StructField("Thriller", IntegerType(), True),
    StructField("War", IntegerType(), True),
    StructField("Western", IntegerType(), True)
])

# Cargar el archivo de películas en un DataFrame, con separador |
dfPeliculas = spark.read.csv("dbfs:/FileStore/u.item", sep="|", schema=esquemaPeliculas, header=False)

# Mostrar las 10 películas con más votos
dfRatingsNombres = dfRatings.join(dfPeliculas,on="MovieID",how="inner")
dfRatingsAgrupados = dfRatingsNombres.groupBy("Title").agg(F.count("Title").alias("Ratings")).orderBy("Ratings",ascending=False)
dfRatingsAgrupados.show(10)


+--------------------+-------+
|               Title|Ratings|
+--------------------+-------+
|    Star Wars (1977)|    583|
|      Contact (1997)|    509|
|        Fargo (1996)|    508|
|Return of the Jed...|    507|
|    Liar Liar (1997)|    485|
|English Patient, ...|    481|
|       Scream (1996)|    478|
|    Toy Story (1995)|    452|
|Air Force One (1997)|    431|
|Independence Day ...|    429|
+--------------------+-------+
only showing top 10 rows



### 10. union() o unionAll():
Se utiliza para combinar dos DataFrames con las mismas columnas. `union()` elimina duplicados, `unionAll()` no.  
    https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html
y https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.unionAll.html

In [0]:
# Crear dos dataframes con los mismos campos (películas más veces puntuadas y películas mejor puntuadas)
data_best = [
    ("Inception", 200, 8.5),
    ("Avatar", 150, 7.8),
    ("Titanic", 180, 7.9)
]
dfRatingsBest = spark.createDataFrame(data_best, ["Title", "Ratings", "MeanRating"])

data_top = [
    ("The Dark Knight", 220, 9.0),
    ("Inception", 200, 8.5),
    ("Interstellar", 130, 8.6)
]
dfRatingsTop = spark.createDataFrame(data_top, ["Title", "Ratings", "MeanRating"])

# Hacer la unión y mostrarlos.
dfUnion = dfRatingsBest.union(dfRatingsTop)
dfUnion.show()

# Películas más veces puntuadas
# dfRatingsBest (Title, Ratings, MeanRating)

# Películas mejor puntuadas, con más de 100 votos
# dfRatingsTop (Title, Ratings, MeanRating)

# dfRatingsBest.union(dfRatingsTop).show()



+---------------+-------+----------+
|          Title|Ratings|MeanRating|
+---------------+-------+----------+
|      Inception|    200|       8.5|
|         Avatar|    150|       7.8|
|        Titanic|    180|       7.9|
|The Dark Knight|    220|       9.0|
|      Inception|    200|       8.5|
|   Interstellar|    130|       8.6|
+---------------+-------+----------+



### 11. map():
Se utiliza para aplicar una función a cada fila del DataFrame, convirtiéndolo a RDD.  
La función map() se aplica a RDDs (Resilient Distributed Datasets), no directamente a DataFrames. Para usar map en un DataFrame, primero debes convertirlo a un RDD usando df.rdd.   
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html


In [0]:
# Obtener una lista de nombres de películas y un diccionario con el número de votos en cada puntuación:

# Nos quedamos con las columnas MovieID y Rating
dfRatings = dfRatings.select("MovieID", "Rating")

# Convertir el DataFrame de ratings a un RDD de filas
rddFilas = dfRatings.rdd

# Convertir el RDD de filas a un RDD de tuplas
rddTuplas = rddFilas.map(lambda fila: (fila[0], (fila[1],1)))

# Función para crear un diccionario con el número de votos para cada puntuación
def crearRatingDict(tuplas):
    RatingDict = {}
    for rating, cont in tuplas:
        if rating in RatingDict:
            RatingDict[rating] += cont
        else:
            RatingDict[rating] = cont
    return RatingDict

# Agrupar por MovieID y agregar las puntuaciones
rddRatingsAgrupados = rddTuplas.groupByKey().mapValues(crearRatingDict)

# Volver a convertir a dataframe y hacer join con películas para obtener el nombre
dfRatingDict = rddRatingsAgrupados.toDF(["MovieID", "RatingDict"])
dfJoined = dfPeliculas.join(dfRatingDict, on="MovieID", how="inner")

# Mostrar 10 películas con su nombre y puntuaciones
dfJoined.select("Title", "RatingDict").show(10, truncate=False)
